<a href="https://colab.research.google.com/github/sominshim/map/blob/main/EDA_%EB%B3%B5%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Visualization HW #4 17011720 심소민

Due date: Dec 02 Wed

data: [Chicago Crimes - 2001 to present](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present-Dashboard/5cd6-ry5g)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.core.display import display, HTML, Javascript
from string import Template
import pandas as pd
import numpy as np
import json, random
import IPython.display
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

In [3]:
filename = '/content/drive/My Drive/Crimes_-_2001_to_Present.csv'
data = pd.read_csv(filename)
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11034701,JA366925,01/01/2001 11:00:00 AM,016XX E 86TH PL,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,412,4.0,8.0,45.0,11,NaN,NaN,2001,08/05/2017 03:50:08 PM,NaN,NaN,NaN
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,2222,22.0,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,835,8.0,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,313,3.0,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,122,1.0,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN


경범죄와 중범죄 구분 기준
: 어떤 범주로 형벌이 가해지는지 [FindLaw](https://www.findlaw.com/), [FreeAdvice](https://www.freeadvice.com/)등 사이트를 참고하였다. 정도에 따라 달라지는 것은 rough하게 often/general한 경우로 취급하였다. 그 외의 언급이 없는 경우, 경계가 애매하므로 애매함 카테고리로 분류한다.

In [4]:
Felony = ['CRIMINAL SEXUAL ASSAULT', 'BURGLARY', 'SEX OFFENSE', 'CRIM SEXUAL ASSAULT', 'SEX OFFENSE', 'NARCOTICS', 
          'ROBBERY', 'ARSON', 'KIDNAPPING', 'HUMAN TRAFFICKING']
Vagueness = ['OFFENSE INVOLVING CHILDREN', 'MOTOR VEHICLE THEFT', 'GAMBLING', 'STALKING', 'OTHER NARCOTIC VIOLATION', 
             'CONCEALED CARRY LICENSE VIOLATION', 'RITUALISM']
Misdemeanor = ['DECEPTIVE PRACTICE', 'THEFT', 'CRIMINAL DAMAGE', 'OTHER OFFENSE', 'BATTERY', 'ASSAULT', 'CRIMINAL TRESPASS', 
               'WEAPONS VIOLATION', 'OBSCENITY', 'LIQUOR LAW VIOLATION', 'PROSTITUTION', 'NON-CRIMINAL', 
               'PUBLIC PEACE VIOLATION', 'INTIMIDATION', 'INTERFERENCE WITH PUBLIC OFFICER', 'PUBLIC INDECENCY', 
               'NON-CRIMINAL (SUBJECT SPECIFIED)', 'DOMESTIC VIOLENCE']
HOMICIDE = ['HOMICIDE']

In [5]:
len(Felony)+len(Vagueness)+len(Misdemeanor)+len(HOMICIDE)==len(data['Primary Type'].unique())

True

In [6]:
data['Felony'] = 0
data.loc[data['Primary Type'].isin(Felony),'Felony'] = 1
data['Vagueness'] = 0
data.loc[data['Primary Type'].isin(Vagueness),'Vagueness'] = 1
data['Misdemeanor'] = 0
data.loc[data['Primary Type'].isin(Misdemeanor),'Misdemeanor'] = 1
data['HOMICIDE'] = 0
data.loc[data['Primary Type'].isin(HOMICIDE),'HOMICIDE'] = 1
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Felony,Vagueness,Misdemeanor,HOMICIDE
0,11034701,JA366925,01/01/2001 11:00:00 AM,016XX E 86TH PL,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,412,4.0,8.0,45.0,11,NaN,NaN,2001,08/05/2017 03:50:08 PM,NaN,NaN,NaN,0,0,1,0
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,2222,22.0,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN,1,0,0,0
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,835,8.0,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN,1,0,0,0
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,313,3.0,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN,0,0,1,0
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,122,1.0,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN,1,0,0,0


In [7]:
crimes = data.copy()
crimes['Date'] = pd.to_datetime(crimes.Date, format='%m/%d/%Y %I:%M:%S %p')
# crimes.index = pd.DatetimeIndex(crimes.Date1)

crimes=crimes.sort_values(by='Date', ascending=True)
crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Felony,Vagueness,Misdemeanor,HOMICIDE
1784561,1651806,G437744,2001-01-01,031XX N SEMINARY AV,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,1932,19.0,NaN,NaN,06,1168442.0,1920901.0,2001,08/17/2015 03:03:40 PM,41.938485,-87.656344,"(41.938484782, -87.656343697)",0,0,1,0
1551623,1353509,G057402,2001-01-01,034XX W ADAMS ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,1123,11.0,NaN,NaN,11,1153613.0,1898824.0,2001,08/17/2015 03:03:40 PM,41.878212,-87.711432,"(41.87821192, -87.711432075)",0,0,1,0
3132857,3361926,HK408994,2001-01-01,059XX N CLARK ST,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,False,2012,20.0,40.0,77.0,02,1164565.0,1939287.0,2001,03/31/2006 10:03:38 PM,41.989020,-87.670069,"(41.989019862, -87.67006944)",0,1,0,0
3357765,3661792,HK760148,2001-01-01,054XX S DREXEL AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,2131,2.0,5.0,41.0,06,1183119.0,1869343.0,2001,03/31/2006 10:03:38 PM,41.796676,-87.604013,"(41.796676399, -87.604012947)",0,0,1,0
1564351,1370284,G074254,2001-01-01,003XX N MASON AV,0820,THEFT,$500 AND UNDER,PARKING LOT/GARAGE(NON.RESID.),False,False,1512,15.0,NaN,NaN,06,1136750.0,1901645.0,2001,08/17/2015 03:03:40 PM,41.886272,-87.773282,"(41.886271732, -87.773282434)",0,0,1,0


In [8]:
crimes['year']=crimes.Date.dt.year
crimes['month']=crimes.Date.dt.month

In [9]:
crimes['crime_type'] = crimes['Misdemeanor']
crimes.loc[crimes['Felony']==1 ,'crime_type'] = 2
crimes.loc[crimes['HOMICIDE']==1 ,'crime_type'] = 3
# crimes.head()

In [10]:
crimes['Mis_sum'] = ((crimes['Arrest']==False) & (crimes['crime_type']==1)).cumsum()
crimes['Fel_sum'] = ((crimes['Arrest']==False) & (crimes['crime_type']==2)).cumsum()
crimes['Hom_sum'] = ((crimes['Arrest']==False) & (crimes['crime_type']==3)).cumsum()

crimes['Fel/Mis'] = (crimes['Fel_sum']/crimes['Mis_sum'])
crimes['Hom/Fel'] = (crimes['Hom_sum']/crimes['Fel_sum'])

# crimes['year'] = crimes['year'].astype('string')
# crimes['month'] = crimes['month'].astype('string')

In [11]:
crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Felony,Vagueness,Misdemeanor,HOMICIDE,year,month,crime_type,Mis_sum,Fel_sum,Hom_sum,Fel/Mis,Hom/Fel
1784561,1651806,G437744,2001-01-01,031XX N SEMINARY AV,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,1932,19.0,NaN,NaN,06,1168442.0,1920901.0,2001,08/17/2015 03:03:40 PM,41.938485,-87.656344,"(41.938484782, -87.656343697)",0,0,1,0,2001,1,1,1,0,0,0.0,NaN
1551623,1353509,G057402,2001-01-01,034XX W ADAMS ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,1123,11.0,NaN,NaN,11,1153613.0,1898824.0,2001,08/17/2015 03:03:40 PM,41.878212,-87.711432,"(41.87821192, -87.711432075)",0,0,1,0,2001,1,1,2,0,0,0.0,NaN
3132857,3361926,HK408994,2001-01-01,059XX N CLARK ST,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,False,2012,20.0,40.0,77.0,02,1164565.0,1939287.0,2001,03/31/2006 10:03:38 PM,41.989020,-87.670069,"(41.989019862, -87.67006944)",0,1,0,0,2001,1,0,2,0,0,0.0,NaN
3357765,3661792,HK760148,2001-01-01,054XX S DREXEL AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,2131,2.0,5.0,41.0,06,1183119.0,1869343.0,2001,03/31/2006 10:03:38 PM,41.796676,-87.604013,"(41.796676399, -87.604012947)",0,0,1,0,2001,1,1,3,0,0,0.0,NaN
1564351,1370284,G074254,2001-01-01,003XX N MASON AV,0820,THEFT,$500 AND UNDER,PARKING LOT/GARAGE(NON.RESID.),False,False,1512,15.0,NaN,NaN,06,1136750.0,1901645.0,2001,08/17/2015 03:03:40 PM,41.886272,-87.773282,"(41.886271732, -87.773282434)",0,0,1,0,2001,1,1,4,0,0,0.0,NaN


In [12]:
crimes.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'Felony', 'Vagueness', 'Misdemeanor', 'HOMICIDE', 'year',
       'month', 'crime_type', 'Mis_sum', 'Fel_sum', 'Hom_sum', 'Fel/Mis',
       'Hom/Fel'],
      dtype='object')

In [25]:
df3 = crimes[['Date','Arrest', 'Ward', 'Latitude', 'Longitude',
       'Felony', 'Vagueness', 'Misdemeanor', 'HOMICIDE', 'year','month', 'crime_type']]
df3.reset_index(inplace=True)
df3

,index,Date,Arrest,Ward,Latitude,Longitude,Felony,Vagueness,Misdemeanor,HOMICIDE,year,month,crime_type
0,1784561,2001-01-01 00:00:00,False,NaN,41.938485,-87.656344,0,0,1,0,2001,1,1
1,1551623,2001-01-01 00:00:00,False,NaN,41.878212,-87.711432,0,0,1,0,2001,1,1
2,3132857,2001-01-01 00:00:00,False,40.0,41.989020,-87.670069,0,1,0,0,2001,1,0
3,3357765,2001-01-01 00:00:00,False,5.0,41.796676,-87.604013,0,0,1,0,2001,1,1
4,1564351,2001-01-01 00:00:00,False,NaN,41.886272,-87.773282,0,0,1,0,2001,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7227295,418750,2020-11-04 23:45:00,False,3.0,NaN,NaN,0,0,1,0,2020,11,1
7227296,418857,2020-11-04 23:49:00,True,6.0,NaN,NaN,0,0,1,0,2020,11,1
7227297,419023,2020-11-04 23:51:00,False,28.0,NaN,NaN,0,0,1,0,2020,11,1
7227298,419179,2020-11-04 23:56:00,True,4.0,NaN,NaN,0,0,1,0,2020,11,1


In [26]:
df3['Fel_w'] = pd.DataFrame(df3['Felony'].groupby(df3['Ward']).cumsum())
df3['Mis_w'] = pd.DataFrame(df3['Misdemeanor'].groupby(df3['Ward']).cumsum())
df3['Hom_w'] = pd.DataFrame(df3['HOMICIDE'].groupby(df3['Ward']).cumsum())

In [27]:
df3 = df3.dropna(axis=0)
df3.reset_index(inplace=True)

In [28]:
df3.columns

Index(['level_0', 'index', 'Date', 'Arrest', 'Ward', 'Latitude', 'Longitude',
       'Felony', 'Vagueness', 'Misdemeanor', 'HOMICIDE', 'year', 'month',
       'crime_type', 'Fel_w', 'Mis_w', 'Hom_w'],
      dtype='object')

In [29]:
df3 = pd.DataFrame(df3.groupby(['year', 'month', 'Ward', 'crime_type'])['Date', 'Arrest', 'Ward', 'Latitude', 'Longitude', 'crime_type', 'Fel_w', 'Mis_w', 'Hom_w'].head(1))
df3.reset_index(inplace=True)

In [32]:
df3[df3['Date'] == '2001-01-01 00:00:00']

,index,Date,Arrest,Ward,Latitude,Longitude,crime_type,Fel_w,Mis_w,Hom_w
0,0,2001-01-01,False,40.0,41.989020,-87.670069,0,0,0,0
1,1,2001-01-01,False,5.0,41.796676,-87.604013,1,0,1,0
2,2,2001-01-01,False,17.0,41.763415,-87.629872,1,0,1,0
3,3,2001-01-01,False,7.0,41.746697,-87.549015,2,1,0,0
4,4,2001-01-01,False,43.0,41.919153,-87.636402,1,0,1,0
5,5,2001-01-01,False,26.0,41.899291,-87.707551,1,0,1,0
6,6,2001-01-01,False,8.0,41.751610,-87.585331,1,0,1,0
7,7,2001-01-01,False,6.0,41.765054,-87.641686,1,0,1,0
8,8,2001-01-01,False,27.0,41.897476,-87.711355,2,1,0,0
9,10,2001-01-01,False,24.0,41.848345,-87.706223,1,0,1,0


In [24]:
df3[df3['Ward']=1]

,index,Date,Arrest,Ward,Latitude,Longitude,crime_type,Fel_w,Mis_w,Hom_w
45,60,2001-01-01 00:00:00,False,1.0,41.901776,-87.670977,1,0,1,0
97,205,2001-01-01 09:00:00,False,1.0,41.894144,-87.680510,0,0,3,0
99,208,2001-01-01 09:00:00,False,1.0,41.910197,-87.688296,2,1,3,0
177,461,2001-02-07 12:00:00,False,1.0,41.935498,-87.682793,1,1,5,0
371,791,2001-04-01 00:01:00,False,1.0,41.920758,-87.692882,1,1,7,0
...,...,...,...,...,...,...,...,...,...,...
40220,6515394,2020-09-02 20:33:00,True,1.0,41.912837,-87.694618,3,19893,99831,109
40247,6517256,2020-09-05 22:00:00,False,1.0,41.893483,-87.658354,0,19900,99860,109
40308,6531473,2020-10-01 02:53:00,False,1.0,41.913411,-87.699179,0,19934,100047,110
40334,6531556,2020-10-01 09:30:00,False,1.0,41.895018,-87.675495,1,19934,100048,110


In [31]:
df3.pivot_table(values='Mis_w', index='Ward', columns=['Ward'], fill_value = 0)

crime_type                   0  ...                   3
Date       2001-01-01 00:00:00  ... 2020-11-04 16:07:00
Ward                            ...                    
1.0                          0  ...                   0
2.0                          0  ...                   0
3.0                          0  ...                   0
4.0                          1  ...                   0
5.0                          0  ...                   0
6.0                          2  ...                   0
7.0                          0  ...                   0
8.0                          1  ...                   0
9.0                          0  ...                   0
10.0                         0  ...                   0
11.0                         0  ...                   0
12.0                         0  ...                   0
13.0                         0  ...                   0
14.0                         0  ...                   0
15.0                         2  ...                   0
16.0                         0  ...                   0
17.0                         3  ...                   0
18.0                         0  ...                   0
19.0                         0  ...                   0
20.0                         0  ...                   0
21.0                         0  ...                   0
22.0                         0  ...                   0
23.0                         0  ...                   0
24.0                         1  ...                   0
25.0                         0  ...               72283
26.0                         0  ...                   0
27.0                         0  ...                   0
28.0                         0  ...                   0
29.0                         0  ...                   0
30.0                         1  ...                   0
31.0                         0  ...                   0
32.0                         0  ...                   0
33.0                         0  ...                   0
34.0                         2  ...                   0
35.0                         0  ...                   0
36.0                         0  ...                   0
37.0                         0  ...                   0
38.0                         0  ...                   0
39.0                         0  ...                   0
40.0                         0  ...                   0
41.0                         0  ...                   0
42.0                         0  ...                   0
43.0                         0  ...                   0
44.0                         0  ...                   0
45.0                         0  ...                   0
46.0                         0  ...                   0
47.0                         0  ...                   0
48.0                         0  ...                   0
49.0                         1  ...                   0
50.0                         0  ...                   0

[50 rows x 27628 columns]

In [18]:
crimes = crimes.dropna(axis=0)

In [1]:
pd.DataFrame(crimes.groupby(['year', 'month', 'Ward', 'crime_type'])['Date', 'Latitude', 'Longitude','crime_type'].head(1))

NameError: ignored

In [ ]:
df3.pivot_table(values='Mis_w', index='Ward', columns=['Date'], fill_value = 0)

In [ ]:
df.Date

0          01/01/2001 11:00:00 AM
1          10/08/2017 03:00:00 AM
2          03/28/2017 02:00:00 PM
3          09/09/2017 08:17:00 PM
4          08/26/2017 10:00:00 AM
                    ...          
7227295    05/26/2019 05:13:00 PM
7227296    05/26/2019 06:48:00 AM
7227297    07/01/2014 07:30:00 AM
7227298    11/30/2017 09:00:00 AM
7227299    05/26/2019 02:11:00 AM
Name: Date, Length: 7227300, dtype: object

In [ ]:
df1 = crimes[['Date', 'Ward', 'Latitude', 'Longitude',
            'crime_type', 'Mis_sum', 'Fel_sum', 'Hom_sum', 
              'Fel/Mis', 'Hom/Fel']]
df1.head()

,Date,Ward,Latitude,Longitude,crime_type,Mis_sum,Fel_sum,Hom_sum,Fel/Mis,Hom/Fel
Date,,,,,,,,,,
2001-01-01,2001-01-01,NaN,41.938485,-87.656344,1,1,0,0,0.0,NaN
2001-01-01,2001-01-01,NaN,41.878212,-87.711432,1,2,0,0,0.0,NaN
2001-01-01,2001-01-01,40.0,41.989020,-87.670069,0,2,0,0,0.0,NaN
2001-01-01,2001-01-01,5.0,41.796676,-87.604013,1,3,0,0,0.0,NaN
2001-01-01,2001-01-01,NaN,41.886272,-87.773282,1,4,0,0,0.0,NaN


In [ ]:
pd.concat([df1, df2])

In [ ]:
crimes

In [ ]:
df1 = crimes.dropna(axis=0)
df2 = pd.DataFrame(df1.groupby(['year', 'month'])['Mis_sum'].head(1))

In [ ]:
ward = df2['Ward'].unique()
len(ward)

48

In [ ]:
ward = df1['Ward'].unique()
len(ward)

50

In [ ]:
df1['District'].unique()

array([19., 11., 20.,  2., 15., 22.,  7.,  3.,  4., 18.,  5.,  9., 10.,
       25., 12.,  8., 17., 24., 14., 16.,  6.,  1., 31., 21., nan])

In [ ]:
df1[df1['Ward'].isnull()]

,Date,Arrest,District,Ward,Community Area,Latitude,Longitude,crime_type
Date,,,,,,,,
2001-01-01 00:00:00,2001-01-01 00:00:00,False,19.0,NaN,NaN,41.938485,-87.656344,1
2001-01-01 00:00:00,2001-01-01 00:00:00,False,11.0,NaN,NaN,41.878212,-87.711432,1
2001-01-01 00:00:00,2001-01-01 00:00:00,False,15.0,NaN,NaN,41.886272,-87.773282,1
2001-01-01 00:00:00,2001-01-01 00:00:00,False,15.0,NaN,NaN,41.886272,-87.773282,1
2001-01-01 00:00:00,2001-01-01 00:00:00,False,22.0,NaN,NaN,41.702754,-87.636510,1
...,...,...,...,...,...,...,...,...
2020-07-25 06:33:00,2020-07-25 06:33:00,False,16.0,NaN,76.0,42.006074,-87.927365,1
2020-08-08 10:30:00,2020-08-08 10:30:00,False,16.0,NaN,10.0,42.000208,-87.806768,1
2020-08-17 16:30:00,2020-08-17 16:30:00,False,24.0,NaN,1.0,42.019386,-87.673147,1


In [ ]:
df1=df1.sort_index(ascending=True)
df1.head()

,Date,Arrest,Ward,Latitude,Longitude,Felony,Misdemeanor,HOMICIDE,Mis_sum,Fel/Mis,Fel_sum,Hom/Fel
Date,,,,,,,,,,,,
2001-01-01,2001-01-01,False,NaN,41.938485,-87.656344,0,1,0,1147010,0.250599,160658,0.015150
2001-01-01,2001-01-01,False,NaN,41.878212,-87.711432,0,1,0,1016245,0.235391,140490,0.015218
2001-01-01,2001-01-01,False,40.0,41.989020,-87.670069,0,0,0,1902930,0.300940,284796,0.030229
2001-01-01,2001-01-01,False,5.0,41.796676,-87.604013,0,1,0,2031618,0.304616,304958,0.029204
2001-01-01,2001-01-01,False,NaN,41.886272,-87.773282,0,1,0,1022634,0.237187,141581,0.015221
